In [10]:
# Copyright 2025 Forusone

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Agent Development Kit - Sequential Agents

### Install adk

In [11]:
%pip install --upgrade --quiet google-adk

In [12]:
# @title Project Information

PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"

MODEL_LIST = ["gemini-2.0-flash", \
              "gemini-2.5-pro-preview-03-25", \
              "gemini-2.5-flash-preview-04-17"]

MODEL_NAME = MODEL_LIST[2]
print(MODEL_NAME)

gemini-2.5-flash-preview-04-17


In [13]:
# @title Authentication to access to GCP
import sys
from pprint import pprint

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

## Initialize Vertex AI

* Please refer to configuration to **initialize Vertex AI : [Link](https://google.github.io/adk-docs/get-started/quickstart/#set-up-the-model)**
* You can set up api key rather than usning Vertex AI.


In [14]:
import os

GOOGLE_GENAI_USE_VERTEXAI = "TRUE"                            #@param {type:"string"}
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = GOOGLE_GENAI_USE_VERTEXAI

GOOGLE_CLOUD_PROJECT = "ai-hangsik"                           #@param {type:"string"}
os.environ["GOOGLE_CLOUD_PROJECT"] = GOOGLE_CLOUD_PROJECT

GOOGLE_CLOUD_LOCATION = "us-central1"                         #@param {type:"string"}
os.environ["GOOGLE_CLOUD_LOCATION"] = GOOGLE_CLOUD_LOCATION


## adk with tools

In [15]:
import asyncio

from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types


In [16]:
positive_agent = LlmAgent(
    name="PositiveAgent",
    model=MODEL_NAME,
    instruction="""You are a helpful AI assistant to find positive side of given technology
                   Add the title "## Positive aspects" to the response.

    """,
    description="Find positive aspects for the given technology",
    output_key="positive_perspectives"
)

negative_agent = LlmAgent(
    name="NegativeAgent",
    model=MODEL_NAME,
    instruction="""You are a helpful AI assistant to find negative side of given technology
                   Add the title "## Negative aspects" to the response.
    """,
    description="Find negative aspects for the given technology",
    output_key="negative_perspectives",
)

conculusion_agent = LlmAgent(
    name="ConclusionAgent",
    model=MODEL_NAME,
    instruction="""You are a helpful AI assistant to draw conclusion by reasoning the both side of positive and negative
                   Add the title "## Conclusion" to the response.
    """,
    description="Draw conculusion.",
    output_key="conculusion",
)

In [17]:
from google.adk.agents import SequentialAgent

pipeline_agent = SequentialAgent(
    name="PipelineAgent",
    sub_agents=[positive_agent, negative_agent, conculusion_agent],
    description="Executes a sequence of positive_agent, negative_agent, and refactoring.",
)


In [18]:
app_name = "Assistant"
user_id = "shins"

query = "what is Generative AI?"

session_service = InMemorySessionService()
session = session_service.create_session(app_name=app_name, user_id=user_id)

runner = Runner(agent=pipeline_agent,
                app_name=session.app_name,
                session_service=session_service)

content = types.Content(role='user', parts=[types.Part(text=query)])

events = runner.run(user_id=session.user_id,
                    session_id=session.id,
                    new_message=content)

for event in events:
    print(event.content.parts[0].text)

    # if event.is_final_response():
    #     final_response = event.content.parts[0].text
    #     print(f"Agent Response: {final_response}\n")
    # else:
    #     print("No results")

## Positive aspects

Generative AI is a type of artificial intelligence that is designed to create new content, rather than just analyzing or acting on existing data. It learns patterns and structures from vast datasets and uses that understanding to generate novel outputs such as text, images, music, code, and more.

Here are some of the key positive aspects of Generative AI:

*   **Boosts Creativity and Innovation:** It can act as a co-creator, helping humans brainstorm ideas, generate multiple versions of content quickly, and explore possibilities that might not have been considered otherwise. This accelerates creative processes in design, art, writing, music, and other fields.
*   **Increases Efficiency and Automation:** By automating the creation of drafts, summaries, boilerplate code, reports, marketing copy, and other content, Generative AI significantly reduces the time and effort required for many tasks, freeing up humans for more complex and strategic work.
*   **Enhances Per

## End of notebook